In [1]:
import pandas as pd
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

from pathlib import Path
# k_CurrentDir  = Path(__file__).parent    # __file__ is not known in Jupyter context
k_Current_dir   = Path.cwd()
k_AssetsDir     = "assets"
k_Gold          = 1.618                    # gold number for ratio
k_Width         = 12
k_Height        = k_Width/k_Gold
k_WidthPx       = 1024
k_HeightPx      = k_WidthPx/k_Gold
k_random_state  = 42
k_test_size     = 0.3
k_num_words     = 1_000                     # the number of most freq words to keep during tokenization
k_epochs        = 50                       # I tried 10, 20, 50 and 100


c:\Users\phili\anaconda3\envs\att\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# -----------------------------------------------------------------------------
# drop empty cols and duplicates, rename cols...
def cleaner(df):
    df.drop(columns="Unnamed: 2", inplace=True)
    df.drop(columns="Unnamed: 3", inplace=True)
    df.drop(columns="Unnamed: 4", inplace=True)

    df.drop_duplicates(inplace=True)

    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace("/", "_")

    df.rename(columns={"v1": "labels"}, inplace=True)
    df.rename(columns={"v2": "texts"}, inplace=True)

    df["labels"] = df["labels"].map({"ham":0,"spam":1})


    return df

In [3]:
df = pd.read_csv(k_Current_dir / k_AssetsDir / "spam.csv", encoding="cp1252")
df = cleaner(df)
labels = df['labels'].tolist()
texts = df['texts'].tolist()



In [4]:

# Charger le tokenizer et le modèle BERT pré-entraîné
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# max_len = long max d'un sms
def encode_texts(texts, tokenizer, max_len=100):
    return tokenizer(
        texts,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )

input_ids = Input(shape=(100,), dtype=tf.int32, name="input_ids")
attention_masks = Input(shape=(100,), dtype=tf.int32, name="attention_mask")

# Extraction des caractéristiques avec BERT
embeddings = bert_model(input_ids, attention_mask=attention_masks)[0]

# Ajout d'une couche de classification
cls_token = embeddings[:, 0, :]
output = Dense(1, activation='sigmoid')(cls_token)

# Définir le modèle
model = Model(inputs=[input_ids, attention_masks], outputs=output)

# Compiler le modèle
model.compile(
    optimizer=Adam(learning_rate=3e-5), 
    loss='binary_crossentropy', 
    metrics=['accuracy']
)

# Résumé du modèle
model.summary()


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 100)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 100,                                           

In [5]:
# path = Path(f"{k_Current_dir/k_AssetsDir/'bert_arch.png'}")
# tf.keras.utils.plot_model(model, path, show_shapes=True)

In [6]:
# Encoder les textes avec BERT tokenizer
encoded_data = encode_texts(texts, tokenizer, max_len=100)

# Division des données
X_train, X_test, y_train, y_test = train_test_split(
    {
        'input_ids': encoded_data['input_ids'], 
        'attention_mask': encoded_data['attention_mask']
    },
    labels, test_size=0.2, random_state=42)

# Entraînement
history = model.fit(
    [X_train['input_ids'], X_train['attention_mask']],
    y_train, 
    validation_data=([X_test['input_ids'], X_test['attention_mask']], y_test),
    batch_size=32,
    epochs=3
)



ValueError: Found input variables with inconsistent numbers of samples: [2, 5169]